# Instalación de librerías

Lo primero es instalar las librerías necesarias para realizar la tarea.

In [1]:
# Instalamos las librerías necesarias
%pip install scrapy bs4 elasticsearch

Note: you may need to restart the kernel to use updated packages.


# Compilar datos del documento web

Se va a crear un Crawler para extraer información de la página de Bulbapedia. Conretamente, el crawler va a extraer la información de todos lo Pokémon, empezará por un primer enlace (Abomasnow) y seguirá con los enlaces de la página para obtener la información del resto. 

Vamos a crear este crawler en un archivo `.py` a parte ([`crawler.py`](crawler.py#L15)).

Una vez tenemos creado el crawler, vamos a ejecutarlo para obtener los datos. Los resultados de cada pokemon se guardarán en un archivo JSON cuyo nombre será el número de la pokédex del pokemon.

In [2]:
from crawler import BulbaSpyder
import scrapy, os
from scrapy.crawler import CrawlerProcess

# Creamos un proceso de Crawler podemos poner distintas settings que están definidas en la documentación.
# Entre ellas podemos ocular los logs del proceso de Crawling.
process = CrawlerProcess(settings={
    "LOG_ENABLED": False,
    # Used for pipeline 1
})

# Comprobamos que existe la carpeta y si no existe la creamos
if not os.path.exists('bulbapedia'):
    os.mkdir('bulbapedia')

# Creamos el proceso con el RSSSpider
process.crawl(BulbaSpyder)
# Ejecutamos el Crawler
process.start()

{
    "url": "https://bulbapedia.bulbagarden.net/wiki/Abomasnow_(Pok%C3%A9mon)",
    "pokemon": "Abomasnow",
    "pokedex_number": "0460",
    "origin": "Abomasnow appears to be based on a cross between an evergreen tree covered in snow and a yeti. It may also be based on 樹氷 juhyou, trees completely covered in snow known as \"snow monsters\". It also resembles a snowy owl.",
    "description": "Abomasnow is a large, bipedal Pokémon covered in shaggy, white fur. It has pale purple eyes with bushy eyebrows and long strands of fur covering its mouth. Two tufts of longer fur grow on both its back and chest, with the chest fur being longer on the female. Its hands, feet, and tail are dark green and spiky, similar to the foliage of an evergreen tree. On its back are four green spikes that resemble pinecones. Abomasnow lives in snowy mountains and appears only when snow flowers bloom. Once the flowers die, it retreats to isolated areas because Abomasnow prefers to be alone and not associate w

# Buscador

Importamos las librerías necesarias.

In [3]:
from elasticsearch import Elasticsearch, helpers
import uuid

Nos conectamos a Elasticsearch.

In [4]:
# Conexión a Elasticsearch
client = Elasticsearch(['http://localhost:9200'])

def obtener_informacion_cluster():
    try:
        # Obtener información sobre el clúster de Elasticsearch
        info_cluster = client.cluster.health()
        print("Información del clúster:")
        print(info_cluster)
    except Exception as e:
        print("Error al obtener información del clúster:", e)

# Llamar a la función para obtener información del clúster
obtener_informacion_cluster()

Información del clúster:
{'cluster_name': 'docker-cluster', 'status': 'yellow', 'timed_out': False, 'number_of_nodes': 1, 'number_of_data_nodes': 1, 'active_primary_shards': 27, 'active_shards': 27, 'relocating_shards': 0, 'initializing_shards': 0, 'unassigned_shards': 2, 'delayed_unassigned_shards': 0, 'number_of_pending_tasks': 0, 'number_of_in_flight_fetch': 0, 'task_max_waiting_in_queue_millis': 0, 'active_shards_percent_as_number': 93.10344827586206}


Creamos el índice.

In [5]:
# Nos conectamos a Elasticsearch
es = Elasticsearch(
    [{'host': 'localhost', 'port': 9200, 'scheme': 'http'}],
    verify_certs=False
)

# Nombramos el índice
index_name = 'bulbapedia_index'

# Borramos el índice si ya existe
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)

# Creamos el índice
es.indices.create(
    index=index_name,
    body={
        "mappings": {
            "properties": {
                "url": {"type": "keyword"},
                "name": {"type": "keyword"},
                "origin": {"type": "text"},
                "description": {"type": "text"},
            }
        }
    }
)

ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'bulbapedia_index'})

Cargamos los datos en Elasticsearch.

In [6]:
import json
import os
from pathlib import Path
from pprint import pprint

# Creamos una lista para guardar los documentos
documents: list = [] 

# Directorio donde se encuentran los archivos JSON
directory: Path = Path('bulbapedia')

# Creamos un bucle que recorra los archivos JSON de la carpeta bulbapedia
for filename in os.listdir(directory):
    if filename.endswith('.json'):
        file_path = directory / filename
        with open(file_path, 'r', encoding='utf-8') as file:
            data = json.load(file)
            document = {
                "_index": index_name,
                "_id": uuid.uuid4(),
                "_source": {
                    "url": data["url"],
                    "number": data["pokedex_number"],
                    "name": data["pokemon"],
                    "origin": data["origin"],
                    "description": data["description"]
                },
            }
            documents.append(document)

# Insertamos los documentos en Elasticsearch
helpers.bulk(es, documents)

(501, [])

In [7]:
import time

# Comprobamos que se han insertado los documentos
time.sleep(1)
count = es.count(index=index_name)['count']
print(f"Number of documents in index '{index_name}': {count}")

Number of documents in index 'bulbapedia_index': 501
